<a href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/docs/notebooks/conditional_text_generation_with_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Generating Synthetic Text

This notebook will walk you through generating realistic but synthetic text examples using an open-source implementation of OpenAI's GPT-3 architecture. 

In this example, we will generate new annotated text utterances that can be used to augment a real world financial dataset called `banking77`. This augmented dataset will have additional annotated examples that can help downstream ML models better understand and respond to new customer queries. To run this notebook, you will need an API key from the Gretel console,  at https://console.gretel.cloud. 
<br>

** **Limitations and Biases** **
Large-scale language models such as GPT-X may produce untrue and/or offensive content without warning. We recommend having a human curate or filter the outputs before releasing them, both to censor undesirable content and to improve the quality of the results. For more information and examples please see [OpenAI](https://huggingface.co/gpt2#limitations-and-bias) and [EleutherAI](https://huggingface.co/EleutherAI/gpt-neo-125M#limitations-and-biases)'s docs for more details.

In [1]:
%%capture
!pip install -U gretel-client

## Set up your project
* `DATASET_PATH`: Specify a dataset to run on.
* `INTENT`: Select an intent from the training data to boost examples for.
* `SEPARATOR`: Specify a separator character (default=`,`) to combine intents and texts with into a single column.
* `PROJECT`: Specify a project name.

In [2]:
import json

import pandas as pd
from gretel_client import configure_session
from gretel_client.helpers import poll
from gretel_client.projects import create_or_get_unique_project, get_project


DATASET_PATH = 'https://gretel-public-website.s3.us-west-2.amazonaws.com/datasets/banking77.csv'
INTENT = "card arrival"
SEPARATOR = ','
PROJECT = 'banking77'

In [3]:
# Log into Gretel and configure project

configure_session(api_key="prompt", cache="yes", endpoint="https://api.gretel.cloud", validate=True, clear=True)

project = create_or_get_unique_project(name=PROJECT)
project

Gretel Api Key··········
Caching Gretel config to disk.
Using endpoint https://api.gretel.cloud
Logged in as alex@gretel.ai ✅


## Create the model configuration

In this notebook we will use GPT-Neo, a transformer model designed using EleutherAI's replication of OpenAI's GPT-3 Architecture. This model has been pre-trained on the Pile, a large-scale dataset using 300 billion tokens over 572,300 steps. In this introductory example, we will fine-tune GPT-Neo to generate synthetic text utterances for a given intent that could be used to train a chat-bot.

In [23]:
config = {
  "schema_version": 1,
  "models": [
    {
      "gpt_x": {
        "data_source": "__",
        "pretrained_model": "EleutherAI/gpt-neo-125M",
        "batch_size": 4,
        "epochs": 1,
        "weight_decay": 0.1,
        "warmup_steps": 100,
        "lr_scheduler": "cosine",
        "learning_rate": 1e-6
      }
    }
  ]
}

## Load and preview the training dataset
Create single-column CSV training set by combining `intent` + `SEPARATOR` + `text`.


In [18]:
def create_dataset(dataset_path: str) -> pd.DataFrame:
    """
    Combine intents and text into a single string to pass to GPT-X.
    """
    records = []
    max_tokens = 0
    
    df = pd.read_csv(dataset_path)
    df['intent_and_text'] = df['intent'] + SEPARATOR + df['text']
    return df
    

pd.set_option('max_colwidth', None)

df = create_dataset(DATASET_PATH)
df[['intent_and_text']].to_csv('finetune.csv', index=False)
df

,intent,text,intent_and_text
0,card arrival,I am still waiting on my card?,"card arrival,I am still waiting on my card?"
1,card arrival,What can I do if my card still hasn't arrived after 2 weeks?,"card arrival,What can I do if my card still hasn't arrived after 2 weeks?"
2,card arrival,I have been waiting over a week. Is the card still coming?,"card arrival,I have been waiting over a week. Is the card still coming?"
3,card arrival,Can I track my card while it is in the process of delivery?,"card arrival,Can I track my card while it is in the process of delivery?"
4,card arrival,"How do I know if I will get my card, or if it is lost?","card arrival,How do I know if I will get my card, or if it is lost?"
...,...,...,...
9998,country support,You provide support in what countries?,"country support,You provide support in what countries?"
9999,country support,What countries are you supporting?,"country support,What countries are you supporting?"
10000,country support,What countries are getting support?,"country support,What countries are getting support?"
10001,country support,Are cards available in the EU?,"country support,Are cards available in the EU?"


## Train the synthetic model
In this step, we will task the worker running in the Gretel cloud, or locally, to fine-tune the GPT language model on the source dataset.

In [19]:
%%time 

model = project.create_model_obj(model_config=config)
model.data_source = "finetune.csv"
model.name = f"{PROJECT}-gpt"
model.submit_cloud()

poll(model)

INFO: Starting poller


{
    "uid": "628e6d41a57288116691dca6",
    "guid": "model_29fPd1kGOhdGRZjbMOMc2qHOZOo",
    "model_name": "banking77-gpt",
    "runner_mode": "cloud",
    "user_id": "60ec86ce492fbf1c604a6ea5",
    "user_guid": "user_26U3XeNlVkqbJkvxJp0vcBs0LJQ",
    "billing_domain": "gretel.ai",
    "billing_domain_guid": "domain_28bzIokk1eQdWUYsovba0VN1gtY",
    "project_id": "628e54742502122db83ba5ee",
    "project_guid": "proj_29fCl4Oi1C657qskW33hBLZgqyF",
    "status_history": {
        "created": "2022-05-25T17:54:09.571482Z"
    },
    "last_modified": "2022-05-25T17:54:09.742768Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "container_image": "074762682575.dkr.ecr.us-east-2.amazonaws.com/models/gpt_x@sha256:645c9aa274c2762db703817b9335f32b24892adb9086fa0b93de51ec91cc6ada",
    "model_type": "gpt_x",
    "config": {
        "schema_version": "1.0",
        "name": "banking77-gpt",
 

INFO: Status is created. Model creation has been queued.
INFO: Status is pending. A Gretel Cloud worker is being allocated to begin model creation.
INFO: Status is active. A worker has started creating your model!
2022-05-25T17:54:18.745378Z  Training gpt_x!
2022-05-25T17:56:29.606013Z  Training in progress
{
    "loss": 4.4147,
    "learning_rate": 0.0,
    "epoch": 1.0
}
2022-05-25T17:56:33.499169Z  Training in progress
{
    "eval_loss": 4.019423484802246,
    "eval_runtime": 3.89,
    "eval_samples_per_second": 514.135,
    "eval_steps_per_second": 32.133,
    "epoch": 1.0
}
2022-05-25T17:56:36.328866Z  Training in progress
{
    "train_runtime": 116.5773,
    "train_samples_per_second": 68.65,
    "train_steps_per_second": 2.145,
    "train_loss": 4.41467041015625,
    "epoch": 1.0
}
2022-05-25T17:56:36.334387Z  Training is completed!
2022-05-25T17:56:36.346104Z  Saving model
2022-05-25T17:56:37.271216Z  Model has been created successfully
2022-05-25T17:57:29.187138Z  Uploading ar

CPU times: user 966 ms, sys: 106 ms, total: 1.07 s
Wall time: 3min 28s


## Generate synthetic text data
The next cells walk through sampling data from the fine-tuned model using a prompt (conditional data generation). 

In [40]:
# Generate new text examples for a given intent by seeding
# model generation with examples from the class. Hint: We have found
# prompting the model with ~25 examples for the class you wish to 
# generate to work well in practice.

def create_prompt(df: pd.DataFrame, intent: str = "", recs: int = 25) -> str:
    """
    Seed GPT-X text generation with an intent from the training data.
    """
    sample = df.query(f'intent == "{intent}"').head(recs)
    prompt = "\n".join([x[0] for x in sample[['intent_and_text']].values])
    return prompt


prompt = create_prompt(df=df, intent=INTENT, recs=25)

record_handler = model.create_record_handler_obj(
    params={"num_records": 1, 
            "maximum_text_length": 1000, 
            "prompt": prompt}
)
record_handler.submit_cloud()
poll(record_handler)

INFO: Starting poller


{
    "uid": "628e73180d41322b2204c714",
    "guid": "model_run_29fSenZRBCtOj66ctg8dwd70V90",
    "model_name": null,
    "runner_mode": "cloud",
    "user_id": "60ec86ce492fbf1c604a6ea5",
    "user_guid": "user_26U3XeNlVkqbJkvxJp0vcBs0LJQ",
    "billing_domain": "gretel.ai",
    "billing_domain_guid": "domain_28bzIokk1eQdWUYsovba0VN1gtY",
    "project_id": "628e54742502122db83ba5ee",
    "project_guid": "proj_29fCl4Oi1C657qskW33hBLZgqyF",
    "status_history": {
        "created": "2022-05-25T18:19:03.970000Z"
    },
    "last_modified": "2022-05-25T18:19:04.113000Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "container_image": "074762682575.dkr.ecr.us-east-2.amazonaws.com/models/gpt_x@sha256:4dcb24f2e01a1ceba52e025f3fc9cfd63a2fff14b2d7da94ace8fbddf09ceae2",
    "model_id": "628e6d41a57288116691dca6",
    "model_guid": "model_29fPd1kGOhdGRZjbMOMc2qHOZOo",
    "action": "gpt

INFO: Status is created. A job has been queued.
INFO: Status is pending. A Gretel Cloud worker is being allocated
INFO: Status is active. A worker has started!
2022-05-25T18:21:13.654124Z  Sampling 1 records conditioned on 'card arrival,I am still waiting on my card?
card arrival,What can I do if my card still hasn't arrived after 2 weeks?
card arrival,I have been waiting over a week. Is the card still coming?
card arrival,Can I track my card while it is in the process of delivery?
card arrival,How do I know if I will get my card, or if it is lost?
card arrival,When did you send me my new card?
card arrival,Do you have info about the card on delivery?
card arrival,What do I do if I still have not received my new card?
card arrival,Does the package with my card have tracking?
card arrival,I ordered my card but it still isn't here
card arrival,Why has my new card still not come?
card arrival,I still haven't received my card after two weeks, is it lost?
card arrival,Can you track my card 

# Creating synthetic intents

In the cell below, we process the raw texts generated by GPT-X into a structured dataframe format, by splitting each row based on the intent prefix (`card_arrival`) that was used to prompt generation.



In [41]:
def get_intents(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extract new intents generated by the GPT-X model.
    """
    MIN_LENGTH = 20
    texts = []
    
    for idx, row in gptx_df.iterrows(): 
        for text in row[0].split(f"{INTENT}{SEPARATOR}"):
            text = text.strip()
            if len(text) > MIN_LENGTH:
                texts.append([INTENT, text])

    intents = pd.DataFrame(texts, columns=['intent', 'synthetic_text'])
    return intents


gptx_df = pd.read_csv(record_handler.get_artifact_link("data"), compression='gzip')
syn = get_intents(df=gptx_df)
syn.head(15)


,intent,synthetic_text
0,card arrival,How do I know if my card still isn't coming?
1,card arrival,How do I know if my card has been returned yet?
2,card arrival,Why does the card I just issued still have no tracking information?
3,card arrival,Have you checked the status of my card?
4,card arrival,Is there a way to check your card status?
5,card arrival,What is your new card's tracking number?
6,card arrival,I have no tracking number for my card yet. Do I have a tracking number?
7,card arrival,Are there any other information about my card that I can track?
8,card arrival,Do I have an extra card with a tracking number?
9,card arrival,"If I don't have any of your tracking information, then I am going to get another card."
